# CatboostClassifier

##Load data and libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold,StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
!pip install catboost --quiet
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, normalize, StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score
import time
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 65.7MB 47kB/s 


In [ ]:
!git clone https://github.com/rijulb17/Job_A_Thon_AV_hackathon.git

Cloning into 'Job_A_Thon_AV_hackathon'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [ ]:
train = pd.read_csv("/content/Job_A_Thon_AV_hackathon/train.csv")
test = pd.read_csv("/content/Job_A_Thon_AV_hackathon/test.csv")

In [ ]:
train_df = train.copy()
test_df = test.copy()

## Data preparation

In [ ]:
train_df = train_df.drop(['ID'], axis =1)
test_df = test_df.drop(['ID'], axis =1)

In [ ]:
x=[]
for i in test_df.columns:
  x.append(i)
x=np.array(x)
x

array(['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration',
       'Holding_Policy_Type', 'Reco_Policy_Cat', 'Reco_Policy_Premium'],
      dtype='<U23')

In [ ]:
# To Keep only Categorical features 
index_to_del = [4,5,11]

In [ ]:
x = np.delete(x,index_to_del)

In [ ]:
categorical_feats = x

In [ ]:
# To have all categorical features in string format
for i in x:
  train_df[i] = train_df[i].astype('str')
  test_df[i] = test_df[i].astype('str')

In [ ]:
def cleaning_df(data):
    #handling missing values
    data.Holding_Policy_Duration = data.Holding_Policy_Duration.replace(np.nan,0)
    data.Holding_Policy_Type = data.Holding_Policy_Type.fillna(0)
    data['Health Indicator'] = data['Health Indicator'].replace(np.nan,'X0') 
        
    return data

In [ ]:
def to_scale(data,X,y):
  # Scaling 
  scaler = MinMaxScaler()
  np_scaled = scaler.fit_tranform(X)
  X = pd.DataFrame(np_scaled)

  return data

In [ ]:
# Function for training and validation

def classify(model, X_tr, y_tr, X_t, X_v, y_v):
  # Model fit and accuracy on validation set calculation
  model.fit(X_tr,y_tr, use_best_model = True, cat_features = categorical_feats)
  y_v_pred_probs  = model.predict_proba(X_v)[:,1]
  y_v_pred_values = model.predict(X_v)

  acc_score_v = accuracy_score(y_v,y_v_pred_values)

  roc_score_v = roc_auc_score(y_v,y_v_pred_probs) 

  # To get predictions on X_test
  #y_prob = model.predict_proba(X_test)[:,1]

  return acc_score_v , roc_score_v #, y_prob


# CatboostClassifier

###Model 1 - Raw data

In [ ]:
X = train_df.drop(['Response'], axis =1)
y = train_df['Response']

In [ ]:
cat = CatBoostClassifier(cat_features=categorical_feats)

In [ ]:
cat.fit(X,y, cat_features=categorical_feats)

Learning rate set to 0.055162
0:	learn: 0.6722371	total: 119ms	remaining: 1m 59s
1:	learn: 0.6549361	total: 185ms	remaining: 1m 32s
2:	learn: 0.6377910	total: 251ms	remaining: 1m 23s
3:	learn: 0.6251737	total: 316ms	remaining: 1m 18s
4:	learn: 0.6135436	total: 350ms	remaining: 1m 9s
5:	learn: 0.6051889	total: 371ms	remaining: 1m 1s
6:	learn: 0.5954750	total: 415ms	remaining: 58.8s
7:	learn: 0.5874428	total: 448ms	remaining: 55.6s
8:	learn: 0.5797833	total: 489ms	remaining: 53.8s
9:	learn: 0.5731943	total: 530ms	remaining: 52.5s
10:	learn: 0.5678080	total: 579ms	remaining: 52.1s
11:	learn: 0.5553882	total: 629ms	remaining: 51.8s
12:	learn: 0.5441308	total: 678ms	remaining: 51.5s
13:	learn: 0.5343972	total: 739ms	remaining: 52s
14:	learn: 0.5260154	total: 787ms	remaining: 51.6s
15:	learn: 0.5194117	total: 840ms	remaining: 51.6s
16:	learn: 0.5136030	total: 888ms	remaining: 51.3s
17:	learn: 0.5117735	total: 949ms	remaining: 51.8s
18:	learn: 0.5069479	total: 998ms	remaining: 51.5s
19:	learn

In [ ]:
X_test = test_df

In [ ]:
y_prob = cat.predict_proba(X_test)

In [ ]:
y_pred = y_prob[:,1]

In [ ]:
y_pred

array([0.03350741, 0.15983289, 0.44432784, ..., 0.01331795, 0.52913216,
       0.06681877])

In [24]:
y_pred = np.array( y_pred)

submission = pd.DataFrame({
        "ID": test["ID"],
        "Response": y_pred
    })
submission.to_csv('./solution.csv', index=False)